# Simple Linear Regression


In [59]:
!pip3 install statsmodel

Exception:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/pip/basecommand.py", line 215, in main
    status = self.run(options, args)
  File "/usr/lib/python3/dist-packages/pip/commands/install.py", line 353, in run
    wb.build(autobuilding=True)
  File "/usr/lib/python3/dist-packages/pip/wheel.py", line 749, in build
    self.requirement_set.prepare_files(self.finder)
  File "/usr/lib/python3/dist-packages/pip/req/req_set.py", line 380, in prepare_files
    ignore_dependencies=self.ignore_dependencies))
  File "/usr/lib/python3/dist-packages/pip/req/req_set.py", line 554, in _prepare_file
    require_hashes
  File "/usr/lib/python3/dist-packages/pip/req/req_install.py", line 278, in populate_link
    self.link = finder.find_requirement(self, upgrade)
  File "/usr/lib/python3/dist-packages/pip/index.py", line 465, in find_requirement
    all_candidates = self.find_all_candidates(req.name)
  File "/usr/lib/python3/dist-packages/pip/index.py", line 423, in fin

#### Loading modules.

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


ImportError: No module named abc

In [14]:
#Reading the csv file.
salary = pd.read_csv('resources/salary.csv')
salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514 entries, 0 to 513
Data columns (total 11 columns):
salary         513 non-null float64
exprior        514 non-null int64
yearsworked    514 non-null int64
yearsrank      514 non-null int64
market         514 non-null float64
degree         514 non-null int64
otherqual      514 non-null int64
position       514 non-null int64
male           514 non-null int64
Field          514 non-null int64
yearsabs       514 non-null int64
dtypes: float64(2), int64(9)
memory usage: 44.2 KB


In [18]:
# Filling my NaN value with the mean of the column
salary = salary.apply(lambda x: x.fillna(x.mean()))

In [19]:
salary.head()

,salary,exprior,yearsworked,yearsrank,market,degree,otherqual,position,male,Field,yearsabs
0,53000.0,0,0,0,1.17,1,0,1,1,3,0
1,58000.0,1,0,0,1.24,1,0,1,1,2,0
2,45500.0,0,0,0,1.21,1,0,1,1,3,2
3,35782.0,0,2,1,0.99,1,0,1,1,4,1
4,34731.0,0,2,2,0.91,1,0,1,1,4,1


In [60]:

# Creating the target variable for the test and training sets of data
X = salary.drop('yearsworked', axis = 1)
y = salary['salary']
X.info()
# Splitting the data into test and training sets by 75/25
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, random_state = 42)
#print(X_train.shape)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514 entries, 0 to 513
Data columns (total 10 columns):
salary       514 non-null float64
exprior      514 non-null int64
yearsrank    514 non-null int64
market       514 non-null float64
degree       514 non-null int64
otherqual    514 non-null int64
position     514 non-null int64
male         514 non-null int64
Field        514 non-null int64
yearsabs     514 non-null int64
dtypes: float64(2), int64(8)
memory usage: 40.2 KB


In [61]:
missing_values = X_train.isnull().sum().sum()
print('Number of missing values:',missing_values)

highest =  X_train.max()
print('The highest value is: ', highest)

('Number of missing values:', 0)
('The highest value is: ', salary       91405.00
exprior         25.00
yearsrank       28.00
market           1.33
degree           1.00
otherqual        1.00
position         3.00
male             1.00
Field            4.00
yearsabs       118.00
dtype: float64)


In [71]:
model = LinearRegression().fit(X, y)
model.fit(1,20)

ValueError: Expected 2D array, got scalar array instead:
array=1.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [63]:
r_sq = model.score(X, y)
print('coefficient of determination:', r_sq)

('coefficient of determination:', 1.0)


In [64]:
print('intercept:', model.intercept_)
print('slope:', model.coef_)

('intercept:', -1.4551915228366852e-11)
('slope:', array([ 1.00000000e+00, -7.81149470e-14, -1.25094958e-13, -2.40791820e-13,
        1.19802793e-13, -6.68114772e-14, -1.32475283e-12,  1.38791440e-12,
        2.91183113e-13,  2.58078888e-14]))


In [69]:
# y_pred = model.predict(X)
# print('predicted response:', y_pred)
model.predict([1,20])

ValueError: Expected 2D array, got 1D array instead:
array=[ 1 20].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [66]:
y_pred = model.intercept_ + model.coef_ * X
>>> print('predicted response:', y_pred)

('predicted response:',           salary       exprior     yearsrank        market        degree  \
0    53000.00000 -1.455192e-11 -1.455192e-11 -1.483364e-11 -1.443211e-11   
1    58000.00000 -1.463003e-11 -1.455192e-11 -1.485050e-11 -1.443211e-11   
2    45500.00000 -1.455192e-11 -1.455192e-11 -1.484327e-11 -1.443211e-11   
3    35782.00000 -1.455192e-11 -1.467701e-11 -1.479030e-11 -1.443211e-11   
4    34731.00000 -1.455192e-11 -1.480211e-11 -1.477104e-11 -1.443211e-11   
5    56000.00000 -1.463003e-11 -1.455192e-11 -1.483846e-11 -1.443211e-11   
6    61008.00000 -1.455192e-11 -1.480211e-11 -1.487217e-11 -1.443211e-11   
7    39986.00000 -1.455192e-11 -1.455192e-11 -1.479030e-11 -1.443211e-11   
8    36308.00000 -1.455192e-11 -1.467701e-11 -1.475659e-11 -1.443211e-11   
9    55000.00000 -1.455192e-11 -1.455192e-11 -1.484809e-11 -1.443211e-11   
10   55227.00000 -1.455192e-11 -1.492720e-11 -1.484809e-11 -1.443211e-11   
11   55227.00000 -1.455192e-11 -1.492720e-11 -1.484809e-11 -1.44